## NAPARI

In [ ]:
import utils    

import napari
import dask
import dask.array as da
import numpy as np
import os
import glob

from natsort import natsorted
from napari_animation import Animation

os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()

import cv2

In [ ]:
def split_label_dims(img_path):
    img = cv2.imread(img_path,0)
    final_image = np.zeros_like(img)

    for value in [1, 2, 3, 4]:
        single_channel_img = np.where(img == value, 1, 0)
        final_image = np.dstack((final_image, single_channel_img))


    final_image = final_image[:, :, 1:]

    return final_image

# Sort just in case
def import_series(_file_list):
    """ 
    Import series of images as dask array
    _file_list needs to be list of pathts to images that want to be merged.
    check shape at the end.
    """
    
    # lazily read images
    _out_array = []

    for _file in _file_list:
        _z_stack = dask.delayed(cv2.imread)(_file)
        _out_array.append(_z_stack)
 
    # determine shape and type from reference frame
    _ref_array = _out_array[-1].compute()
    _shape = np.shape(_ref_array)
    _dtype = _ref_array.dtype
 
    # convert delayed to dask arrays
    _out_array = [da.from_delayed(i, shape= _shape, dtype= _dtype) for i in _out_array]
   
    # merge dask arrays
    _out_array = da.stack(_out_array)
 
    return _out_array

In [ ]:
# We get the path of the files
# Directory of files to be read, each file needs to be able to fit into ram on its own.
# file_names = glob.glob("./data/66-4/processed_32/*.tif")
he_file_names = glob.glob("./data/66-4/processed_32/clean_dust_bubbles/registered/*.tif")
he_file_names = natsorted(he_file_names)
#stack = import_series(he_file_names)


# predictions = glob.glob("./data/predictions/*.png")
# predictions = natsorted(predictions)

In [ ]:
import matplotlib.pyplot as plt 
import skimage
from tqdm import tqdm_notebook

prediction_paths = glob.glob("./data/predictions/*.png")

labels_to_plot = []
for path in tqdm_notebook(prediction_paths):
    #img = cv2.imread(path, 0)

    img = split_label_dims(path)
    img = img[:,:,0]
    img = skimage.morphology.label(img)
    img = skimage.morphology.remove_small_objects(img, 128)
    labels_to_plot.append(img > 0)
    

labels_to_plot = np.array(labels_to_plot)

stroma = []
for path in tqdm_notebook(prediction_paths):
    #img = cv2.imread(path, 0)

    img = split_label_dims(path)
    img = img[:,:,2]
    stroma.append(img > 0)
    

stroma = np.array(stroma)

fat = []
for path in tqdm_notebook(prediction_paths):
    #img = cv2.imread(path, 0)

    img = split_label_dims(path)
    img = img[:,:,3]
    fat.append(img > 0)
    

fat = np.array(fat)


In [ ]:
predictions_stack = []
for _file in predictions:
    img = cv2.imread(_file, 0)
    predictions_stack.append(img.astype("int64"))
    #predictions_stack.append(split_label_dims(_file))

predictions_stack = np.array(predictions_stack)


he_img = []
for _file in he_file_names:
    img = cv2.imread(_file)
    pad_image = utils._image_operations.white_pad_image(img, 128)
    he_img.append(pad_image)

he_img = np.array(he_img)


In [ ]:
# Predicted images and labels should have the same shape
assert predictions_stack.shape == he_img.shape[:-1]

In [ ]:
labels_to_plot.nbytes*1E-6

In [ ]:
# Look at more things at the same time
viewer = napari.Viewer()
#viewer.add_image(predictions, channel_axis=3, name=['epithelium', 'blood_vessels', "stroma", "adipocytes"])
#viewer.add_image(he_img, scale=(20,8,8))
viewer.add_image(labels_to_plot, scale=(20,8,8), colormap="magenta")
viewer.add_image(stroma, scale=(20,8,8), colormap="cyan")
viewer.add_image(fat, scale=(20,8,8), colormap="yellow")

# Create Videos

Registration animation

In [ ]:
# TODO activate labels with a nice opacity setting to get a smooth video
# TODO fine tune the parameters to get a better video, now it goes a bit fast and not fluid

In [ ]:
animation = Animation(viewer)
viewer.update_console({"animation": animation})
viewer.layers[-1].visible = False
viewer.dims.current_step = (0, 0, 0)
animation.capture_keyframe(steps=1)
viewer.dims.current_step = (100, 0, 0)
animation.capture_keyframe(steps=120)
animation.animate("./videos/animate2D_32.mp4", canvas_only=True)

In [ ]:
animation = Animation(viewer)
viewer.layers[-1].visible = True
viewer.dims.ndisplay = 3
viewer.camera.angles = (0.0, 0.0, 0.0)
animation.capture_keyframe()
viewer.camera.angles = (0.0, 180.0, 0.0)
animation.capture_keyframe(steps=120)
viewer.camera.angles = (0.0, 360.0, 0.0)
animation.capture_keyframe(steps=120)
animation.animate('./videos/test3D_32.mov', canvas_only=True)

# Other stuff

In [ ]:
unaligned = glob.glob("./data/66-4/processed_32/*.tif")
unaligned = natsorted(unaligned)

preprocessed = glob.glob("./data/66-4/processed_32/clean_dust_bubbles/*.tif")
preprocessed = natsorted(preprocessed)

global_transf = glob.glob("./data/66-4/processed_32/clean_dust_bubbles/registered/*.tif")
global_transf = natsorted(global_transf)

elastic_transf = glob.glob("./data/66-4/processed_32/clean_dust_bubbles/registered/elastic registration/*.tif")
elastic_transf = natsorted(elastic_transf)

In [ ]:
test = cv2.imread(global_transf[0])

In [ ]:
# unaligned_stack = []
# for _file in unaligned:
#     img = cv2.imread(_file)
#     img = cv2.resize(img, (test.shape[1], test.shape[0]))
#     pad_image = utils._image_operations.white_pad_image(img, 128)
#     unaligned_stack.append(pad_image)

# unaligned_stack = np.array(unaligned_stack)


# preprocessed_stack = []
# for _file in preprocessed:
#     img = cv2.imread(_file)
#     img = cv2.resize(img, (test.shape[1], test.shape[0]))
#     pad_image = utils._image_operations.white_pad_image(img, 128)
#     preprocessed_stack.append(pad_image)

# preprocessed_stack = np.array(preprocessed_stack)

# global_stack = []
# for _file in global_transf:
#     img = cv2.imread(_file)
#     pad_image = utils._image_operations.white_pad_image(img, 128)
#     global_stack.append(pad_image)

# global_stack = np.array(global_stack)


elastic_stack = []
for _file in elastic_transf:
    img = cv2.imread(_file)
    pad_image = utils._image_operations.white_pad_image(img, 128)
    elastic_stack.append(pad_image)

elastic_stack = np.array(elastic_stack)

In [ ]:
viewer = napari.Viewer()
# viewer.add_image(unaligned_stack, scale=(50,8,8))
# viewer.add_image(preprocessed_stack, scale=(50,8,8))
# viewer.add_image(global_stack, scale=(50,8,8))
viewer.add_image(elastic_stack, scale=(50,8,8))

In [ ]:
# # Quick zarr exploiting way, might not always work
# file_path = r"D:\ERK_large_set\ss_to_MEKi\cytoring_labels.tif"
# img = da.from_zarr(tifffile.imread(file_path, aszarr=True))
# #%%
# # reshape to be able to split channels
# shape = np.shape(img)
# img = img.rechunk((1, shape[-3],shape[-2], shape[-1]))
# #%%
# # view image using napari
# #view = img.swapaxes(0,1)
# napari.view_image(img)

In [ ]:
# import utils._file_operations


# utils._file_operations.create_folder("./data/66-4/reshaped_images")


# check for average image heigh and width
# height = [] 
# width = []
# for file in file_names:
#     img = cv2.imread(file)
#     height.append(img.shape[0])
#     width.append(img.shape[1])

# avg_height = np.round(sum(height)/len(height),0)
# avg_width = np.round(sum(width)/len(width),0)

# for i, file in enumerate(file_names):
#   img = cv2.imread(file)
#   reshaped = cv2.resize(img, (int(avg_height), int(avg_width)), interpolation= cv2.INTER_LINEAR)
#   cv2.imwrite(os.path.join("./data/66-4/reshaped_images","HE_"+str(i)+".tif"), reshaped)